In [1]:
#!pip install -r requirements.txt -q

In [ ]:
import gmpy2
import numpy as np
from functools import partial
from sage.libs.pari.all import pari
from sage.all import ZZ, PolynomialRing, GF

from preprocessing.functions import ParamGen, KeyGen
from preprocessing.encryption import decrypt, encrypt
from preprocessing.message import decode, encode, prepare_slots
from protocols.zpk_o_pk import zk_pok_prover_fiat_shamir, zk_pok_verify_fiat_shamir

SEED = 777
rng = np.random.default_rng(SEED)

pari.allocatemem(2 * 1024**3)

PARI stack size set to 2147483648 bytes, maximum size set to 2147483648


Generar el algebra $A_q$ dado el campo $p^k$ y el tamano del mensaje

In [3]:
n = 3
# 2**64 + 13
p = gmpy2.mpz(2)**16 + 1  # primo que define el cuerpo de base 𝔽_p
k = 1   # queremos incluir 𝔽_{p^k} dentro de A
s = 3   # número de copias de 𝔽_{p^k} que necesitamos
sec = 40

Aq, tau, delta, m, N, coeffs, C_m, q, r = ParamGen(p, k, s, n, sec)

pk, sk, pk_hat = KeyGen(Aq, N, r, q, rng, p)

m = 32768 phi(m) = 16384 degrees: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Encode/Decode messages

In [4]:
print("\n--- Iniciando Prueba de Encoding/Decoding ---")

# 1. Preparar la estructura SIMD
R_p, slots_moduli = prepare_slots(p, coeffs)
num_slots = len(slots_moduli)
print(f"Sistema configurado con {num_slots} slots disponibles.")

# 2. Definir mensajes de prueba
# Solo usaremos 4 valores, el resto serán ceros
plaintext = [ord("H"), ord("o"), ord("l"), ord("a")] 

print(f"Mensajes originales: {plaintext}")

# 3. Codificar (Encode)
plaintext_poly = encode(plaintext, R_p, slots_moduli, Aq)

print("Codificación exitosa. Elemento en Aq creado.")
# print(plaintext_poly) # Descomentar para ver el polinomio gigante

# 4. Decodificar (Decode) para verificar
decoded_message = decode(plaintext_poly, R_p, slots_moduli)

# Filtramos solo los primeros 4 para comparar visualmente
print(f"Mensajes recuperados: {decoded_message[:4]}")

# Verificación automática
assert decoded_message[:4] == plaintext
print("¡Éxito! Los mensajes coinciden.")


--- Iniciando Prueba de Encoding/Decoding ---
Sistema configurado con 16384 slots disponibles.
Mensajes originales: [72, 111, 108, 97]
Codificación exitosa. Elemento en Aq creado.
Mensajes recuperados: [72, 111, 108, 97]
¡Éxito! Los mensajes coinciden.


## Encrypt/Decrypt

In [5]:
# Encryption
ciphertext = encrypt(plaintext, N, r, q, rng, R_p, slots_moduli, Aq, p, pk)

# Decryption
decrypted_messages = decrypt(ciphertext, R_p, slots_moduli, sk)

# Verificación automática
assert decrypted_messages[:4] == plaintext
print("¡Éxito! Los mensajes coinciden.")

¡Éxito! Los mensajes coinciden.


## ZKPoPK

In [ ]:
# Generate parameters for ZKP
chars = list(range(65, 91)) + list(range(97, 123))  # Aa–Zz
messages = rng.choice(chars, size=sec)

witness_xr = []
ciphertexts_c = []

for i, msg in enumerate(messages):
    # avisar cada 5 mensajes (empezando en el 5)
    if (i + 1) % 10 == 0:
        print(f"[INFO] Procesados {i+1} mensajes de {sec}")

    ci, xi, ri = encrypt(plaintext, N, r, q, rng, R_p, slots_moduli, Aq, p, pk, verbose=True)
    witness_xr.append((xi, ri))
    ciphertexts_c.append(ci)


In [ ]:
# Ejecutar ZKP
transcript = zk_pok_prover_fiat_shamir(pk,
                                        ciphertexts_c,
                                        witness_xr,
                                        partial(rng.choice, chars, size=sec),
                                        sec,
                                        N, r, q, rng, R_p, slots_moduli, Aq, p,
                                        serialize_ciphertext=lambda C: str(C).encode())

print("transcript terminado")

ok = zk_pok_verify_fiat_shamir(pk, ciphertexts_c, transcript,
                               N, r, q, rng, R_p, slots_moduli, Aq, p)
print("Verificación:", ok)

## $\Pi_{\text{PREP}}$

In [8]:
from protocols.prep import Pi_prep_init


alpha, alpha_list, beta_list, e_a_list, e_b_list = Pi_prep_init(s, N, r, q, rng, R_p, slots_moduli, Aq, p, pk, k, n)